In [2]:
import torch
import torchvision
import numpy as np

In [3]:
from torchvision.models import *

In [4]:
model = vgg16(pretrained=True)

In [5]:
from attacks import ifgsm

In [6]:
import torchvision.datasets as dset

In [11]:
import torchvision.transforms as transforms

In [12]:
def _data_transforms_cifar10():
  CIFAR_MEAN = [0.49139968, 0.48215827, 0.44653124]
  CIFAR_STD = [0.24703233, 0.24348505, 0.26158768]

  train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
  ])

  valid_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR_MEAN, CIFAR_STD),
    ])
  return train_transform, valid_transform

In [13]:
train_transform, valid_transform = _data_transforms_cifar10()

In [14]:
train_data = dset.CIFAR10(root='../data', train=True, download=True, transform=train_transform)

Files already downloaded and verified


In [15]:
num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(0.5 * num_train))

In [16]:
train_tensors = []
train_label = []

In [17]:
for i in range(num_train):
    train_tensors.append(train_data[i][0])
    train_label.append(train_data[i][1])

In [18]:
print(len(train_tensors))
print(train_tensors[0].size())

50000
torch.Size([3, 32, 32])


In [19]:
num_train = len(train_data)
indices = list(range(num_train))
split = int(np.floor(0.5 * num_train))

train_queue = torch.utils.data.DataLoader(
  train_data, batch_size=64,
  sampler=torch.utils.data.sampler.SubsetRandomSampler(indices[:]),
  pin_memory=True, num_workers=2)

In [20]:
import torch.nn as nn

def ifgsm(model, X, y, niters=10, epsilon=0.01, visualize=False):
    X_pert = X.clone()
    X_pert.requires_grad = True
    
    for _ in range(niters):
        output_perturbed = model(X_pert)
        loss = nn.CrossEntropyLoss()(output_perturbed, y)
        loss.backward()
        pert = epsilon * X_pert.grad.detach().sign()

        # add perturbation
        X_pert = X_pert.detach() + pert
        X_pert.requires_grad = True
        
        # make sure we don't modify the original image beyond epsilon
        X_pert = (X_pert.detach() - X.clone()).clamp(-epsilon, epsilon) + X.clone()
        X_pert.requires_grad = True
        
        # adjust to be within [-1, 1]
        X_pert = X_pert.detach().clamp(-1, 1)
        X_pert.requires_grad = True
        
    return X_pert

In [21]:
from torch.autograd import Variable
perturbed_images = []
perturbed_labels = []
for step, (input, target) in enumerate(train_queue):
    input = Variable(input)
    target = Variable(target)
    
    input = ifgsm(model, input, target, niters=3)
    perturbed_images.append(input)
    perturbed_labels.append(target)

RuntimeError: size mismatch, m1: [64 x 512], m2: [25088 x 4096] at /opt/conda/conda-bld/pytorch_1532579245307/work/aten/src/TH/generic/THTensorMath.cpp:2070

In [ ]:
pert_imgs = torch.cat(perturbed_images)
pert_lbls = torch.cat(perturbed_labels)

In [ ]:
pert_imgs.size()

In [ ]:
pert_lbls.size()

In [ ]:
np.save('vgg16_imgs.npy', pert_imgs.detach().numpy())

In [ ]:
np.save('vgg16_lbls.npy', pert_lbls.detach().numpy())

In [2]:
x = torch.tensor([1])

In [1]:
import torch

In [3]:
x.cuda()

tensor([1], device='cuda:0')

In [4]:
x.numpy()

array([1])

In [7]:
x = Variable(torch.tensor([1])).cuda()

In [6]:
from torch.autograd import Variable

In [10]:
x.cpu().numpy()

array([1])

In [ ]:
x.detach()